# Exploring a new features 

In [2]:
import uproot4
import numpy as np
import pandas as pd
import concurrent.futures

from modules.FourTrackEvents import *

ccup9_2015_file = r'D:\GoogleDrive\Job\cern\Alice\analysis\data\RhoPrime\2015\4ProngsLHC15oMC.root'

tree_name = '4Prongs/events'
# dfs.loc[0] # get dataframe part with entry = 0    

executor = concurrent.futures.ThreadPoolExecutor()

branches = ['T_Px', 'T_Py', 'T_Pz',  'T_Q', 'T_NumberOfSigmaTPCPion', 'T_TPCRefit', 'T_TPCNCls', 'T_Phi', 'T_Eta', 'T_HasPointOnITSLayer0','T_HasPointOnITSLayer1', 'T_ITSModuleInner', 'T_ITSModuleOuter']

evColumns = ["RunNum", "PeriodNumber", "OrbitNumber", "BunchCrossNumber", "Mass", "Pt", "Q", "Rapidity", "Phi", "ZNAenergy", "ZNCenergy", "ZPAenergy", "ZPCenergy", "VtxX", "VtxY", "VtxZ", "VtxContrib", "VtxChi2", "VtxNDF", "SpdVtxX", "SpdVtxY", "SpdVtxZ", "SpdVtxContrib", "V0Adecision", "V0Cdecision", "ADAdecision", "ADCdecision", "V0Afired", "V0Cfired", "ADAfired", "ADCfired", "STPfired", "SMBfired", "SM2fired", "SH1fired", "OM2fired", "OMUfired", "IsTriggered", "nTracklets", "nTracks",'ZDCAtime_0', 'ZDCAtime_1', 'ZDCAtime_2', 'ZDCAtime_3', 'ZDCCtime_0', 'ZDCCtime_1', 'ZDCCtime_2', 'ZDCCtime_3' ] #, "FORChip"]

events = uproot4.open(ccup9_2015_file, object_cache = 5000, num_workers=12, interpretation_executor=executor)[tree_name]

data_tracks = events.arrays(filter_name=branches, library='pd', array_cache=5000)#, entry_stop=1000000)
data_events = events.arrays(filter_name=evColumns, library='pd', array_cache=5000)#, entry_stop=1000000)
chips = events.arrays(filter_name=['FORChip'], library='pd') #, array_cache=5000)#, entry_stop=1000000)
chips = chips.groupby('entry').FORChip.apply(list)

In [9]:
%matplotlib widget 
data_events.Pt.hist(bins=100,range=(0,2), histtype='step')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [10]:
data_events

,RunNum,PeriodNumber,OrbitNumber,BunchCrossNumber,Mass,Pt,Q,Rapidity,Phi,ZNAenergy,...,nTracklets,nTracks,ZDCAtime_0,ZDCAtime_1,ZDCAtime_2,ZDCAtime_3,ZDCCtime_0,ZDCCtime_1,ZDCCtime_2,ZDCCtime_3
0,245407,0,0,0,3.277053,0.067997,-1718026240,-0.202196,3.111930,-157.023392,...,4,4,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
1,245407,0,0,0,1.970822,0.018496,-1718026240,0.389107,3.104819,-336.880157,...,4,4,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
2,245407,0,0,0,2.830512,0.040261,-1718026240,-0.523298,1.994956,-157.023392,...,4,4,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
3,245407,0,0,0,1.805113,0.054843,-1718026240,0.560920,-2.574388,-157.023392,...,4,4,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
4,245407,0,0,0,1.656197,0.056423,-1718026240,0.267238,-1.465889,-157.023392,...,4,4,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9661,246994,0,0,0,1.880020,0.029710,-1718026240,0.017353,2.163878,-157.023392,...,3,4,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
9662,246994,0,0,0,4.866782,0.101007,-1718026240,-0.718959,-0.626682,22.833378,...,1,4,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
9663,246994,0,0,0,1.841085,0.103930,-1718026240,0.194734,-0.392131,-157.023392,...,4,4,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0
9664,246994,0,0,0,2.801142,0.028733,-1718026240,-0.539449,1.793509,22.833378,...,3,4,999.0,999.0,999.0,999.0,999.0,999.0,999.0,999.0


In [51]:
four_tracks_events = np.where(np.array(list(map(lambda x : len(x), data['T_Q'][is_triggered]))) == 4)
zero_charged_events = np.where(np.array(list(map(lambda x : sum(x), data['T_Q'][is_triggered]))) == 0)
four_tracks_zero_charged_events = np.intersect1d(four_tracks_events, zero_charged_events)

In [32]:
data

,RunNum,PeriodNumber,OrbitNumber,BunchCrossNumber,Mass,Pt,Q,Rapidity,Phi,ZNAenergy,...,STPfired,SMBfired,SM2fired,SH1fired,OM2fired,OMUfired,IsTriggered,nTracklets,nTracks,FORChip
0,245145,0,13362961,674,2.802221,0.567083,-1718026238,0.056647,-2.397827,-92.604172,...,True,True,True,True,False,False,False,4,8,"[4, 15, 22, 107, 167, 233, 247, 262, 295, 315,..."
1,245145,3,3674210,3438,2.996301,3.597858,-1718026240,-0.318146,1.097847,-428.223267,...,True,True,True,True,True,False,False,4,6,"[30, 49, 89, 290, 309, 352, 370, 393, 472, 509..."
2,245145,1,12644173,2126,3.198086,1.573138,-1717960705,-0.360621,1.949035,9.372114,...,True,True,True,True,True,True,False,3,5,"[27, 33, 160, 171, 302, 306, 308, 366, 384, 39..."
3,245145,1,16213555,3239,3.103773,0.797095,-1718026238,0.011698,1.207371,291.203796,...,True,True,True,True,True,True,False,12,8,"[12, 29, 33, 36, 50, 71, 168, 215, 268, 284, 3..."
4,245145,1,5525160,2195,1.679243,0.388082,-1718026240,-0.595928,0.879197,344.992584,...,True,True,True,True,False,False,False,27,4,"[0, 7, 28, 31, 37, 38, 39, 45, 46, 55, 57, 71,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106700,246994,3,3847039,3472,5.037951,7.349191,-1717960706,-0.550232,0.569475,-384.684784,...,True,True,True,True,False,False,False,1,6,"[19, 27, 232, 399, 448, 469, 877]"
106701,246994,2,62739,532,8.490516,5.970900,-1717960708,-1.047591,-2.963939,6169.960449,...,True,True,True,True,True,False,False,74,22,"[26, 34, 38, 41, 87, 120, 202, 204, 206, 209, ..."
106702,246994,1,566110,3361,5.639124,0.815031,-1717960707,0.497559,2.903033,-624.092224,...,True,True,True,True,True,True,False,6,5,"[47, 86, 168, 212, 267, 305, 358, 505, 523, 52..."
106703,246994,2,3250243,1830,3.033328,2.172899,-1718026239,0.083928,-2.649607,75394.562500,...,True,True,True,True,True,True,False,4,7,"[49, 163, 203, 248, 287, 353, 452, 472, 584, 6..."


In [3]:
events.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
RunNum               | int32_t                  | AsDtype('>i4')
PeriodNumber         | uint32_t                 | AsDtype('>u4')
OrbitNumber          | uint32_t                 | AsDtype('>u4')
BunchCrossNumber     | uint32_t                 | AsDtype('>u4')
Mass                 | float                    | AsDtype('>f4')
Pt                   | float                    | AsDtype('>f4')
Q                    | int32_t                  | AsDtype('>i4')
Rapidity             | float                    | AsDtype('>f4')
Phi                  | float                    | AsDtype('>f4')
ZNAenergy            | float                    | AsDtype('>f4')
ZNCenergy            | float                    | AsDtype('>f4')
ZPAenergy            | float                    | AsDtype('>f4')
ZPCenergy            | float                    | AsDtype(